<a href="https://colab.research.google.com/github/haalcala/NYP-SDAI/blob/master/ITI105/project/Fraud_detection_harold_knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab" align="left"/></a>

# Fraud detection using kNN

In [1]:
import requests
import io
import os

try:
    os.mkdir("./tmp")
except:
    pass

external_files = [
  {
      "url":"https://raw.githubusercontent.com/haalcala/NYP-SDAI/master/ITI105/project/ds_util.py",
      "local_file":"ds_util.py"
  },
  {
      "url":"https://raw.githubusercontent.com/haalcala/NYP-SDAI/master/ITI105/project/files/bs140513_032310.csv",
      "local_file":"bs140513_032310.csv"
  },
]

for ext_file in external_files:
  req = requests.get(ext_file["url"])

  print(ext_file["local_file"], len(req.content))

  f = open("./tmp/" + ext_file["local_file"],"wb")

  f.write(req.content)

ds_util.py 1917
bs140513_032310.csv 48391391



## load and prepare data


In [2]:
import tmp.ds_util as util

ds_util = util.DSUtil()
ds_util.load_csv("./tmp/bs140513_032310.csv")

# analyse data composition

ds_util.blow_my_mind()

args: {}
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 594643 entries, 0 to 594642
Data columns (total 10 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   step         594643 non-null  int64  
 1   customer     594643 non-null  object 
 2   age          594643 non-null  object 
 3   gender       594643 non-null  object 
 4   zipcodeOri   594643 non-null  object 
 5   merchant     594643 non-null  object 
 6   zipMerchant  594643 non-null  object 
 7   category     594643 non-null  object 
 8   amount       594643 non-null  float64
 9   fraud        594643 non-null  int64  
dtypes: float64(1), int64(2), object(7)
memory usage: 45.4+ MB
None
unique step    180
dtype: int64
unique customer    4112
dtype: int64
unique age    8
dtype: int64
unique gender    4
dtype: int64
unique zipcodeOri    1
dtype: int64
unique merchant    50
dtype: int64
unique zipMerchant    1
dtype: int64
unique category    15
dtype: int64
unique amount    23767


### drop columns not needed for calculations and get dummy variables

In [3]:
import pandas as pd
import numpy as np

df = pd.DataFrame(np.random.randn(1000,1000))

df[df > 0.9] = pd.np.nan

df

C:\Users\Harold\anaconda3\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  


,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,0.012974,-0.122675,0.059026,-0.647534,-1.198372,-1.339966,0.420623,-0.783139,-0.545348,-2.048003,...,NaN,0.352861,-0.302668,NaN,0.608939,-1.268829,-0.137476,-0.182322,NaN,-0.664903
1,-0.071043,-2.104590,NaN,-0.090416,NaN,-1.203617,NaN,-0.393402,0.145450,-0.806814,...,0.405514,NaN,-1.269593,0.758070,-0.934529,-0.072768,0.595768,-1.567491,0.576014,-0.400315
2,NaN,NaN,-0.656532,NaN,NaN,-0.976813,NaN,-0.229003,-0.957063,-0.246518,...,NaN,0.555554,NaN,-0.411919,NaN,NaN,0.118358,NaN,NaN,0.559553
3,-0.068736,-1.539835,0.865991,-0.371779,0.620437,-0.684390,0.030193,0.001365,-0.343344,0.052950,...,-1.134977,NaN,0.329170,-1.670386,-1.021524,0.131671,NaN,NaN,0.094644,NaN
4,-1.426250,-1.057917,-0.424587,0.686743,-1.790493,-0.123119,NaN,0.835416,-1.313486,NaN,...,-0.603197,0.107327,-1.047389,-0.679839,-0.907186,0.547129,0.571047,-0.782650,-0.002479,-1.797154
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,NaN,-0.055144,-0.981281,-0.344716,NaN,0.181353,-0.182296,0.513737,NaN,-0.927110,...,0.218114,NaN,-0.895570,0.663237,0.100245,0.200683,-0.331373,0.583327,-0.543698,0.150964
996,NaN,0.468933,-0.422970,-0.599265,-0.491359,NaN,0.162349,-1.581676,NaN,0.153765,...,0.723607,0.118940,NaN,0.441609,NaN,0.370368,-1.050953,-0.970978,-1.411408,0.454639
997,NaN,0.099493,0.765209,0.716983,0.344330,-0.296416,-0.586814,-0.553714,-0.199113,0.842976,...,0.219337,NaN,-0.879486,0.047322,-0.043341,0.107340,-0.529088,NaN,-1.358040,0.495971
998,NaN,-2.701517,-0.130406,0.317712,NaN,NaN,-0.990247,-0.580490,NaN,0.769269,...,0.576056,-0.911835,-0.153362,-0.311223,-0.714495,0.268929,-0.848210,0.431104,-1.076386,NaN


In [4]:
ds_util.drop_columns(["customer", "zipMerchant", "zipcodeOri"])

df = ds_util.get_dummies()

df

,step,amount,fraud,age_'0',age_'1',age_'2',age_'3',age_'4',age_'5',age_'6',...,category_'es_home',category_'es_hotelservices',category_'es_hyper',category_'es_leisure',category_'es_otherservices',category_'es_sportsandtoys',category_'es_tech',category_'es_transportation',category_'es_travel',category_'es_wellnessandbeauty'
0,0,4.55,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
1,0,39.68,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,0,26.89,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
3,0,17.25,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,0,35.72,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
594638,179,20.53,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
594639,179,50.73,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
594640,179,22.44,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
594641,179,14.46,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0


## kNN Stuffs                                                     


### Training


In [5]:
#We need to import the k-NN Classifier from skleart.neighbors
from sklearn.neighbors import KNeighborsClassifier

target_column = "fraud"

#Set the target column as our label (target value to be predicted)
YTrain = df[target_column]

#Remove the target column from our input variables
XTrain = df.drop(target_column, axis=1)

#Create a k-NN Classifier
classifier = KNeighborsClassifier(n_neighbors=1)

def train_fn():
    #Train the classifier using our training data
    classifier.fit(XTrain, YTrain)

ds_util.activity_wrapper("Training", train_fn)

Training ...
Training ... done. 40.813845 seconds


### Testing


In [6]:
# Get the first fraud row that we will use for testing (this is not recommended!)
XTest_fraud = df.loc[df.fraud==1].drop(target_column, axis=1).iloc[0].values

# Get the first not fraud row that we will use for testing (this is not recommended!)
XTest_not_fraud = df.loc[df.fraud==0].drop(target_column, axis=1).iloc[0].values

# XTest = pd.DataFrame([
#     # this should give value 1 result
#     XTest_fraud,
#     # this should give value 0 result
#     XTest_not_fraud
# ])

# for index, row in XTest.iterrows():
#     print(classifier.predict(row.values.reshape(1, -1)))

total_test_count = 2
tests_count_done = 0
tests_passed = 0
tests_failed = 0

def test_fn():
    global tests_count_done, tests_passed, tests_failed
    
    tests_count_done = tests_count_done+1
    if classifier.predict(XTest_fraud.reshape(1,-1))[0] == 1:
        tests_passed = tests_passed+1
        print("Tested fraud entry to be fraud")
    else:
        tests_failed = tests_failed+1
        print("Failed testing fraud to be fraud")
    
    tests_count_done = tests_count_done+1
    if classifier.predict(XTest_not_fraud.reshape(1,-1))[0] == 0:
        tests_passed = tests_passed+1
        print("Tested not fraud entry to be not fraud")
    else:
        tests_failed = tests_failed+1
        print("Failed testing not fraud to be not fraud")
        
ds_util.activity_wrapper("Testing", test_fn)
     
print(f"Test result: {tests_count_done}/{total_test_count} done. passed: {tests_passed}, failed: {tests_failed}.")

print("All done.")

Testing ...
Tested fraud entry to be fraud
Tested not fraud entry to be not fraud
Testing ... done. 0.010692 seconds
Test result: 2/2 done. passed: 2, failed: 0.
All done.
